# Vislice
V tej nalogi si bomo pogledali slovenske samostalnike s strani igranja igre vislice. Pogledali si bomo predvsem kako izgledajo optimalne strategije in kakšne besede
so vsaj v teoriji najtežje.

Po izbiri pripravite pythonovo virtualno okolje. Preden uporabljate katerekoli programe v tem repozitoriju, poženite naslednje ukaze:

In [ ]:
!pip install -r ../requirements.txt
!python ./data/get_lf_no_lfs.py


## Natančnejši opis problema
### Pravila igre
V splošnem se igro igrata dve osebi. Ena oseba si izmisli besedo, druga pa ugiba črke. Vsakič ko črko pravilno ugotovi, se črko zapiše na njena mesta v besedi.
Če beseda ugibane črke ne vsebuje, oseba ki ugiba izgubi življenje. To se ponavadi označi tako, da druga oseba nariše eno črto na skico obešenega človeka.
Ko je slika končana, je igre konec in zmaga tisti, ki si je besedo izmislil. Če ugotovi vse črke v besedi, zmaga tisti, ki ugiba.

To je splošen opis igre, ki še vedno dopušča mnogo variacij. Za potrebe te naloge se bomo odločili za nekaj točnejših pravil:
* Izbrana beseda je na seznamu besed, ki je znan vnaprej, to so vsi samostalniki iz SSKJ, ki vsebujejo le črke slovenske abecede. Shranjeni so v `data/nouns_si.txt`.
* Male in velike črke uporabljamo za iste črke.
* Ko ugibamo, na začetku poznamo dolžino besede, prvo črko in kje vse se ta črka ponovi.
* Z vidika naloge je število poskusov ko lahko ugibamo nepomembno, zato števila življenj ne bomo določili.

### Optimalna strategija
Kaj je optimalna se lahko spreminja glede na naše potrebe. Za naše potrebe bomo iskali takšno optimalno strategijo:
* Z vsako strategijo bomo pri eni ali več besedah porabili največje število življenj. Če ugibamo to besedo je to najslabši možen primer.
* Strategija je optimalna, če ima v primerjavi z ostalimi strategijami po njihovih najslabših možnih primerih najmanjše število porabljenih življenj.
* Opazovali bomo samo najslabši možen primer. Če imata dve strategiji enako slab najslabši primer, sta enako dobri. Tudi če ena od strategij vse ostale besede reši
z manj izgubljenimi življenji, sta še vedno enako dobri.
* Za strategijo nas zanima število izgubljenih življenj da uganemo besedo v najslabšem možnem primeru. Zato v pravilih ni smiselno določiti po koliko napakah je igre konec.
* Optimalnih strategij je lahko več. Ponavadi jih je celo zelo veliko. 
(Če imamo "se_" - prvi črki sta "s" in "e", ugibamo še tretjo, potem imamo na voljo 10 črk ki dajo besedo in 3628800 različnih strategij)

### Seznam besed
Vir besed je stran [https://fran.si/iskanje?FilteredDictionaryIds=130&View=1&Query=%2A](https://fran.si/iskanje?FilteredDictionaryIds=130&View=1&Query=%2A). 
Program `collector/sskj_collector.py` prebere število strani z besedami in pobere besede z vsake izmed njih. Ima tudi možnost, da se omeji samo na samostalnike.
Potem popravi še vsa naglasna znamenja v navadne črke in pobriše besede ki ne vsebujejo samo slovenskih črk. Na koncu jih uredi po slovenski abecedi.

Besede so že shranjene v datoteki [data/nouns_si.txt](../data/nouns_si.txt), zato jih ni potrebno ponovno nalagati.
Če želite pognati pobiranje besed lahko to storite s pomočjo skripte z ukazom (postopek traja, ker prebere več kot 4600 strani):

In [ ]:
%run ../vislice.py sskjcollect --help
%run ../vislice.py sskjcollect --file data/samostalniki.txt --nounsonly

### Izračun optimalne strategije
Glede na podan seznam besed program `solver/game.py` izračuna najboljšo možno strategijo tako, da sledi korakom:
* Če se v že ugibanih črkah besede ne ujemajo, jih razdelimo na skupine, tako da glede na te črke izgledajo enako. Za vsako skupino izračunamo strategijo posebej in naša strategija ima
toliko napak kot jih ima skupina z največ napakami.
* Če smo besedo uganili, smo zanjo že zmagali. Strategija ima 0 korakov in naredimo 0 napak.
* Če si vse možne besede delijo kakšno črko, ugibamo te črke. Besede razdelimo v skupine, tako da imajo ugibane črke na istih mestih in za vsako od dobljenih skupin
izračunamo najboljšo strategijo. Strategija ima toliko napak kot skupina z največ napakami.
* Za vsako črko, ki je še nismo ugibali, od najbolj do najmanj pogoste poskusimo:
    - Besede razdelimo v skupine, tako da se ujemajo v vseh do zdaj ugibanih črkah in v tej izbrani črki.
    - Izračunamo strategijo za vse skupine. Če besede v tej skupini ne vsebujejo izbrane črke, ima skupina eno napako več kot njena strategija.
    Če besede v skupini vsebujejo črko, ima skupina toliko napak kot ima napak njena strategija.
    - Za najboljšo strategijo vzamemo črko, ki ima pri najslabši skupini najmanjše število napak.
Na koncu dobimo eno izmed najboljših strategij. Strategija je že izračunana in shranjena v [data/nouns_si_strategy_hinted.json](../data/nouns_si_strategy_hinted.json)

Če želite poračunati strategijo lahko to storite s pomočjo ukaza spodaj. (Zelo traja - lahko tudi več kot 8 ur, poraba pomnilnika je do 3 GB.) Spremenite `data/nouns_si.txt`, če želite drug seznam besed.

In [ ]:
%run ../vislice.py hintstrat --help
%run ../vislice.py hintstrat data/nouns_si.txt --output data/strategija.json

## Analiza podatkov


In [8]:
import pandas as pd

strategy = pd.read_json("../data/nouns_si_strategy_hinted.json")
words = strategy["words"]
len(words)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.